# Evaluation of RAG model

One way to think about different types of RAG evaluators is as a tuple of what is being evaluated X what its being evaluated against:

1. Correctness: Response vs reference answer
- Goal: Measure "how similar/correct is the RAG chain answer, relative to a ground-truth answer"
- Mode: Requires a ground truth (reference) answer supplied through a dataset
- Evaluator: Use LLM-as-judge to assess answer correctness.

2. Relevance: Response vs input
- Goal: Measure "how well does the generated response address the initial user input"
- Mode: Does not require reference answer, because it will compare the answer to the input question
- Evaluator: Use LLM-as-judge to assess answer relevance, helpfulness, etc.

3. Groundedness: Response vs retrieved docs
- Goal: Measure "to what extent does the generated response agree with the retrieved context"
- Mode: Does not require reference answer, because it will compare the answer to the retrieved context
- Evaluator: Use LLM-as-judge to assess faithfulness, hallucinations, etc.

4. Retrieval relevance: Retrieved docs vs input
- Goal: Measure "how relevant are my retrieved results for this query"
- Mode: Does not require reference answer, because it will compare the question to the retrieved context
- Evaluator: Use LLM-as-judge to assess relevance

In [9]:
from langchain_google_genai import GoogleGenerativeAI
import google.generativeai as genai
import json
from pydantic import BaseModel
from typing_extensions import Annotated, TypedDict, Type
from langsmith import Client
import os
import pickle as pkl
import helper_functions as hf
from langchain_huggingface import HuggingFaceEmbeddings
import glob
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import ragas

In [3]:
with open('api_google.txt') as f:
    
    api_key = json.load(f)

## Setting

In [4]:
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = api_key['key']

In [10]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai", temperature=0)

In [5]:
with open("generated_questions_final_gemini_2-5.pkl", "rb") as f:
    eval_datasets = pkl.load(f)

## Evaluation

In [6]:
# Grade output schema
class CorrectnessGrade(TypedDict):
    # Note that the order in the fields are defined is the order in which the model will generate them.
    # It is useful to put explanations before responses because it forces the model to think through
    # its final response before generating it:
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    correct: Annotated[bool, ..., "True if the answer is correct, False otherwise."]

# Grade prompt
correctness_instructions = """You are a senior researcher in reprodutive field and bioinformatics. 

You will be given a QUESTION, the GROUND TRUTH (correct) ANSWER, and the PhD Student answer. 

Here is the grade criteria to follow:
(1) Grade the student answers based ONLY on their factual accuracy relative to the ground truth answer. 
(2) Ensure that the student answer does not contain any conflicting statements.
(3) It is OK if the student answer contains more information than the ground truth answer, as long as it is factually accurate relative to the  ground truth answer.

Correctness:
A correctness value of True means that the PhD student's answer meets all of the criteria.
A correctness value of False means that the PhD student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM

grader_llm = llm.with_structured_output(CorrectnessGrade, method="json_schema", strict=True)

In [7]:
# Grade output schema
class RelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "Provide the score on whether the answer addresses the question"]

# Grade prompt
relevance_instructions="""You are a senior researcher in reprodutive field and bioinformatics. 

You will be given a QUESTION and a STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the PhD STUDENT ANSWER is concise and relevant to the QUESTION
(2) Ensure the PhD STUDENT ANSWER helps to answer the QUESTION

Relevance:
A relevance value of True means that the student's answer meets all of the criteria.
A relevance value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

relevance_llm = llm.with_structured_output(RelevanceGrade, method="json_schema", strict=True)

In [8]:
# Grade output schema
class RetrievalRelevanceGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    relevant: Annotated[bool, ..., "True if the retrieved documents are relevant to the question, False otherwise"]

# Grade prompt
retrieval_relevance_instructions = """You are a senior researcher in reprodutive field and bioinformatics grading PhD students questions. 

You will be given a QUESTION and a set of FACTS provided by the PhD student. 

Here is the grade criteria to follow:
(1) You goal is to identify FACTS that are completely unrelated to the QUESTION
(2) If the facts contain ANY keywords or semantic meaning related to the question, consider them relevant
(3) It is OK if the facts have SOME information that is unrelated to the question as long as (2) is met

Relevance:
A relevance value of True means that the FACTS contain ANY keywords or semantic meaning related to the QUESTION and are therefore relevant.
A relevance value of False means that the FACTS are completely unrelated to the QUESTION.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM
retrieval_llm = llm.with_structured_output(RetrievalRelevanceGrade, method="json_schema", strict=True)

In [9]:
# Grade output schema
class GroundedGrade(TypedDict):
    explanation: Annotated[str, ..., "Explain your reasoning for the score"]
    grounded: Annotated[bool, ..., "Provide the score on if the answer hallucinates from the documents"]

# Grade prompt
grounded_instructions = """You are a senior researcher in reprodutive field and bioinformatics grading PhD students questions. 

You will be given FACTS and a PhD STUDENT ANSWER. 

Here is the grade criteria to follow:
(1) Ensure the PhD STUDENT ANSWER is grounded in the FACTS. 
(2) Ensure the PhD STUDENT ANSWER does not contain "hallucinated" information outside the scope of the FACTS.

Grounded:
A grounded value of True means that the student's answer meets all of the criteria.
A grounded value of False means that the student's answer does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct. 

Avoid simply stating the correct answer at the outset."""

# Grader LLM 
grounded_llm = llm.with_structured_output(GroundedGrade, method="json_schema", strict=True)


In [10]:
outputs = {}
for file in glob.glob("./output/*.json"):
    output = json.load(open(file, "r"))
    outputs[re.sub(r'./output/rag_','',file)] = output


In [11]:
try:
    with open("./evaluation2/evaluation.pkl","rb") as f:
        list_dataframes2 = pkl.load(f)
except:
    print('File not found')
    list_dataframes2 = []

File not found


In [13]:
if os.path.exists("./evaluation2/evaluation_log.txt"):
    with open("./evaluation2/evaluation_log.txt", "r") as f:
        lines = [line.strip() for line in f]
else:
    lines = []

for settings, evaluation in tqdm(outputs.items(), total=len(outputs)):

    if settings in lines:
        print(f"Settings: {settings} already analyzed")
        continue
    
    print(f"Analyzing {settings}")
    aux_correct = []
    aux_relevant = []
    aux_retrival = []
    aux_ground = []

    for mydic in evaluation:
        #aux_correct.append(hf.correctness(inputs=mydic, grader_llm = grader_llm, correctness_instructions= correctness_instructions))
        aux_relevant.append(hf.relevance(inputs=mydic, relevance_llm  = relevance_llm, relevance_instructions = relevance_instructions))
        #aux_retrival.append(hf.retrieval_relevance(inputs=mydic, retrieval_llm= retrieval_llm, retrieval_relevance_instructions = retrieval_relevance_instructions))
        #aux_ground.append(hf.groundedness(inputs=mydic, grounded_llm=grounded_llm, grounded_instructions=grounded_instructions))

    # Create dataframe
    # correctness_results = pd.Series(aux_correct, name="Correctness")
    relevance_results = pd.Series(aux_relevant, name="Relevance")
    # retrieval_results = pd.Series(aux_retrival, name="Retrieval")
    # ground_results = pd.Series(aux_ground, name="Groundedness")
    list_dataframes2.append(pd.concat([pd.DataFrame.from_dict(evaluation),relevance_results], axis = 1))

    with open("./evaluation2/evaluation.pkl","wb") as f:
        pkl.dump(list_dataframes2,f)

    with open("./evaluation2/evaluation_log.txt","a") as f:
        f.write(f"{settings}\n")

  0%|          | 0/126 [00:00<?, ?it/s]

Analyzing chunk_1000_reader-model_gemini-2.0-flash_emnedding_model_e5-small-v2recursive_True_k_8.json


  1%|          | 1/126 [00:48<1:40:56, 48.45s/it]

Analyzing chunk_1500_reader-model_gemini-2.0-flash_emnedding_model_snowflake-arctic-embed-srecursive_False_k_9.json


  2%|▏         | 2/126 [01:34<1:36:42, 46.79s/it]

...

 98%|█████████▊| 124/126 [1:41:16<01:35, 47.75s/it]

Analyzing chunk_1000_reader-model_gemini-2.0-flash_emnedding_model_e5-small-v2recursive_False_k_7.json


 99%|█████████▉| 125/126 [1:42:05<00:48, 48.24s/it]

Analyzing chunk_2000_reader-model_gemini-2.0-flash_emnedding_model_e5-small-v2recursive_True_k_7.json


100%|██████████| 126/126 [1:42:54<00:00, 49.00s/it]


## Benchmark

In [5]:
with open("./evaluation/evaluation_final.pkl","rb") as f:
        list_dataframes = pkl.load(f)
with open("./generated_questions_final_gemini_2-5.pkl","rb") as f:
        examples = pkl.load(f)

In [6]:
examples['question'].tolist()[0]

'How does the FGB rs1800790A allele affect fibrinogen levels in F13A 34Val/Val wildtypes?'

In [7]:
template  = """
<|system|>
Give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Now here is the question you need to answer.

Question: {question}
</s>
<|assistant|>
"""

In [78]:
llm.invoke(examples['question'].tolist()[0]).content

'The FGB rs1800790 A allele is associated with **lower** fibrinogen levels, even in individuals with the F13A 34Val/Val genotype.\n\nHere\'s a breakdown of why and how:\n\n* **FGB rs1800790:** This is a single nucleotide polymorphism (SNP) located in the promoter region of the fibrinogen beta chain (FGB) gene.  The two alleles are G and A.  The A allele is generally associated with reduced FGB gene transcription and, consequently, lower plasma fibrinogen levels.\n\n* **F13A 34Val/Val:** This refers to the genotype at the Factor XIIIa (F13A) Val34Leu polymorphism (rs5985).  Val/Val indicates the individual is homozygous for the Valine allele at position 34.  The Leu allele (34Leu) is associated with increased Factor XIIIa activity and potentially a protective effect against thrombosis.  However, in the context of fibrinogen levels, the 34Val/Val genotype is considered the "wildtype" and doesn\'t inherently lower fibrinogen.\n\n* **Interaction:**  The effect of the FGB rs1800790 A allele

In [ ]:
examples_eval = examples.to_dict(orient='records')
llm_reader = init_chat_model("gemini-2.0-flash", model_provider="google_genai",temperature=0.5,max_output_tokens=1024)
output_file_name = f"./output_bench/benchmark.json"

hf.run_rag_tests(
    eval_dataset=examples_eval,
    llm=llm_reader,
    output_file=output_file_name,
    template=template,
    rag=False,
    verbose=True,
    test_settings = "No_RAG",
    recursive_chunk=False
)

In [14]:
sample = ragas.SingleTurnSample(
    user_input = list_dataframes[0].iloc[0,:]['question'],
    response = list_dataframes[0].iloc[0,:]['generated_answer'],
    retrieved_contexts = list_dataframes[0].iloc[0,:]['retrieved_docs']
)

In [47]:
list_dataframes[j]['test_settings'].value_counts()

test_settings
chunk_1000_reader-model_gemini-2.0-flash_emnedding_model_snowflake-arctic-embed-srecursive_False_k_8    63
Name: count, dtype: int64

In [51]:
datasets = []
j = 1

for i in range(0,list_dataframes[j].shape[0]):

    datasets.append(
    {"user_input" : list_dataframes[j].iloc[i,:]['question'],
    "response" : list_dataframes[j].iloc[i,:]['generated_answer'],
    "retrieved_contexts" : list_dataframes[j].iloc[i,:]['retrieved_docs'],
    "reference":list_dataframes[j].iloc[i,:]['true_answer']
    }
    )

len(datasets)

63

In [48]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_list(datasets)
evaluation_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=63)

In [52]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(llm)

from ragas.metrics import LLMContextPrecisionWithoutReference # LLMContextRecall, Faithfulness, FactualCorrectness, 

result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextPrecisionWithoutReference()],llm=evaluator_llm)
result


Evaluating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [00:32<00:00,  1.95it/s]


{'llm_context_precision_without_reference': 0.6748}

In [28]:
scorer.single_turn_ascore(sample)

<coroutine object SingleTurnMetric.single_turn_ascore at 0x7f65250235c0>